In [ ]:
import json
import matplotlib.pyplot as plt
from collections import Counter
import statistics
from datetime import datetime
from collections import defaultdict
from torchtext.vocab import GloVe
from sklearn.feature_extraction.text import CountVectorizer
import re

import torch
from torch import nn
import torch.nn.functional as F
from sklearn.feature_extraction.text import CountVectorizer

from collections import Counter


from sklearn.model_selection import train_test_split



from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

In [ ]:
review_train = []
with open("../dataset/review_train.json", "r") as file:
    for line in file:
        review_train.append(json.loads(line))

review_test = []
with open("../dataset/review_test.json", "r") as file:
    for line in file:
        review_test.append(json.loads(line))


In [ ]:
class NN2DAN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.embedding = read_word_embeddings("data/glove.6B.300d-relativized.txt").get_initialized_embedding_layer()
        self.fc1 = nn.Linear(300, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 2)
        self.log_softmax = nn.LogSoftmax(dim=1)
            
        
    def forward(self, x):
        x = self.embedding(x)
        mask = (x != 0).float()
        x = torch.sum(x, dim=1) / torch.sum(mask, dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.log_softmax(x)
        return x

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_review_text(review_text):
    review_text = review_text.lower()
    review_text = re.sub(r'\W+', ' ', review_text)
    words = [word for word in review_text.split() if word not in stop_words]
    processed_words = [lemmatizer.lemmatize(word, pos='v') for word in words]
    return processed_words

In [ ]:
review_text_train = [
    preprocess_review_text(review['text']) for review in review_train
]

review_text_test = [
    preprocess_review_text(review['text']) for review in review_test
]

In [ ]:
rating_train = [review['stars'] for review in review_train]
rating_test = [review['stars'] for review in review_test]

In [ ]:
class NN2RDAN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.embedding = torch.nn.Embedding(14923, 300)
        
        self.fc1 = nn.Linear(300, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 2)
        self.log_softmax = nn.LogSoftmax(dim=1)
            
    def forward(self, x):
        x = self.embedding(x)
        mask = (x != 0).float()
        x = torch.sum(x, dim=1) / torch.sum(mask, dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.log_softmax(x)
        return x